<a href="https://colab.research.google.com/github/gmohor21/Car-Reviews-Analysis-w-LLMs/blob/dev%2Fcar-review-analysis/Analyzing_Car_Reviews_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


# Introduction and Setup

## Car Review Analysis using Large Language Models (LLMs)

This Jupyter Notebook demonstrates the use of various pre-trained Large Language Models (LLMs) to analyze car reviews. It covers tasks such as sentiment analysis, translation, question answering, and text summarization.

The project is part of a prototype for a chatbot app developed for "Car-ing is sharing", a car sales and rental company. The chatbot aims to address diverse inquiries and provide support to both customers and human agents.

## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

# Install required packages
`!pip install transformers pandas numpy scikit-learn sacrebleu detoxify`

In [ ]:
!pip install transformers
!pip install evaluate

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Start your code here!

# Import required libraries

```import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sacrebleu.metrics import BLEU
from detoxify import Detoxify

In [ ]:
# Import required libraries
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import torch

In [ ]:
import os

# Define the data directory
data_directory = 'data'

# Check if the data directory exists
if os.path.exists(data_directory):
    # List all files in the data directory
    files_in_data_directory = os.listdir(data_directory)
    files_in_data_directory
else:
    f"'{data_directory}' directory does not exist."

In [ ]:
files_in_data_directory

['car_reviews.csv', 'reference_translations.txt']

In [ ]:
# Load the car reviews dataset with proper handling of irregular delimiters
#df = pd.read_csv('data/car_reviews.csv', error_bad_lines=False)

In [ ]:
# Load the dataset
file_path = 'data/car_reviews.csv'
df = pd.read_csv(file_path, sep=';', engine='python')

In [ ]:
df

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [ ]:
df.columns

Index(['Review', 'Class'], dtype='object')

In [ ]:
# Split the single column into two columns
# df[['Review', 'Sentiment']] = df['Review;Class'].str.split(';', expand=True)

In [ ]:
# Drop the original column and null values
# df = df.drop(columns=['Review;Class']).dropna()

In [ ]:
# Map sentiment labels to binary values
df['Class'] = df['Class'].map({'POSITIVE': 1, 'NEGATIVE': 0})

# Task 1: Classify car reviews

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Class'], test_size=0.2, random_state=42)

In [ ]:
# Load pre-trained sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
# Classify sentiments
review_val = [str(i) for i in df['Review'].values]
predicted_labels = sentiment_pipeline(review_val)

In [ ]:
# Map labels to binary values
label_map = {'NEGATIVE': 0, 'POSITIVE': 1}
predictions = [label_map[label['label']] for label in predicted_labels]

In [ ]:
# Calculate metrics
accuracy_result = accuracy_score(df['Class'], predictions)
f1_result = f1_score(df['Class'], predictions, average='binary')

In [ ]:
print(f"Accuracy: {accuracy_result}")
print(f"F1 Score: {f1_result}")

Accuracy: 0.8
F1 Score: 0.8571428571428571


# Task 2: Translate a car review

In [ ]:
# Open the file in read mode
file_path = 'data/reference_translations.txt'

try:
    with open(file_path, 'r') as file:
        # Read the content of the file
        file_content = file.read()

        # Print the content
        print("File Content:\n", file_content)

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

File Content:
 Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.
Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.


In [ ]:
!pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from sacrebleu.metrics import BLEU

In [ ]:
# Extract first two sentences of the first review
first_two_sentences = '. '.join(df['Review'].iloc[0].split('.')[:2]) + '.'

In [ ]:
print(first_two_sentences)

I am very satisfied with my 2014 Nissan NV SL.  I use this van for my business deliveries and personal use.


In [ ]:
# Load English to Spanish translation model
model_name = "Helsinki-NLP/opus-mt-en-es"
translator = pipeline("translation", model=model_name)

In [ ]:
# Translate the text
translated_review = translator(first_two_sentences, max_length=512)[0]['translation_text']

In [ ]:
# Load reference translations
with open('data/reference_translations.txt', 'r', encoding='utf-8') as f:
    reference_translations = [line.strip() for line in f.readlines()]

In [ ]:
# Calculate BLEU score
bleu = BLEU()
bleu_score = bleu.corpus_score([translated_review], [reference_translations]).score

In [ ]:
print(f"Translated Review: {translated_review}")
print(f"BLEU Score: {bleu_score}")

Translated Review: Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.
BLEU Score: 68.88074582865497


# Task 3: Question Answering

In [ ]:
# Perform question answering
#Prepare question and context
question = "What did he like about the brand?"
context = df['Review'].iloc[1]  # Using the second review as context

In [ ]:
# Load QA model
model_name = "deepset/minilm-uncased-squad2"
qa_pipeline = pipeline("question-answering", model=model_name)

In [ ]:
# Get the answer
result = qa_pipeline(question=question, context=context)
answer = result['answer']

In [ ]:
print(f"Context: {context}")
print(f"Question: {question}")
print(f"Answer: {answer}")

Context: The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Question: What did he like about the brand?
Answer: ride quality, reliability


# Task 4: Text Summarization and Toxicity Analysis

## Summarize and analyze a car review

In [ ]:
# Summarize the last review
last_review = df['Review'].iloc[-1]

In [ ]:
from transformers import pipeline

# Load summarization model
# model_name = 'facebook/bart-large-cnn'
# summarizer = pipeline("summarization", model=model_name)
summarizer = pipeline("summarization")
summary = summarizer(last_review, max_length=55, min_length=50, do_sample=False)
summarized_text = summary[0]['summary_text']

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
print(f"Original review: {last_review}")
print(f"Summary: {summarized_text}")

Original review: I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of 

### Analyze toxicity and potential biases of the summary

In [ ]:
from evaluate import load

# Load toxicity metric
toxicity_metric = load("toxicity")
toxicity_scores = toxicity_metric.compute(predictions=[summarized_text])
max_toxicity = max(toxicity_scores['toxicity'])

Downloading:   0%|          | 0.00/816 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
print(f"Maximum Toxicity: {max_toxicity}")

Maximum Toxicity: 0.00013859820319339633


In [ ]:
# Load regard metric

from evaluate import load

regard_metric = load("regard")
regard_scores = regard_metric.compute(data=[summarized_text])

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
print(f"Regard Scores:")
for key, value in regard_scores.items():
    print(f"{key}: {value[0]}")

Regard Scores:
regard: [{'label': 'positive', 'score': 0.7860166430473328}, {'label': 'other', 'score': 0.09952671080827713}, {'label': 'neutral', 'score': 0.08915407955646515}, {'label': 'negative', 'score': 0.025302622467279434}]


In [ ]:
!pip install detoxify

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from detoxify import Detoxify

In [ ]:
# Analyze potential biases
detoxify = Detoxify('original')

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /home/repl/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt


  0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [ ]:
# Calculate toxicity
toxicity_scores = detoxify.predict(summarized_text)
max_toxicity = max(toxicity_scores.values())

In [ ]:
print(f"Maximum Toxicity: {max_toxicity}")

Maximum Toxicity: 0.0005452524637803435
